# Ch01 文本处理工作

该笔记本所用到的库

In [3]:
from importlib.metadata import version

import tiktoken
import torch

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.3.1
tiktoken version: 0.7.0


- 本章介绍数据准备和采样，以便为LLM“准备”输入数据

![Alt text](../../../img/LLM/ch01/LLM_Course_pipeline.png)

# 1、理解word embeddings

- 嵌入有多种形式;在这本书中,我们主要关注文本嵌入

![Alt text](../../../img/LLM/ch01/forms_of_embedding.png)

- LLM与高维空间（即数千维）中的嵌入一起工作
- 由于我们无法可视化这样的高维空间（我们人类在1、2或3维中思考）,下图说明了一个2维嵌入空间

![Alt text](../../../img/LLM/ch01/high_dimensional_space.png)

# 2、Tokenizer text

- 在本节中，我们将文本标记化，这意味着将文本分解为更小的单元，例如单个单词和标点符号

![Alt text](../../../img/LLM/ch01/tokenize_text.png)

- 加载我们要使用的原始文本

In [4]:
import os
import urllib.request

if not os.path.exists('../../../Data/the-verdict.txt'):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [5]:
with open('../../../Data/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

print(f"Total number of article: {len(raw_text)}")
print(raw_text[:99])

Total number of article: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标:tokenize文本为LLM所用
- 基于一些简单的数据开发一个简单的tokenizer,以便于之后我们使用
- 以下正则表达式是将其以空格进行分割

In [6]:
import re

text = "Hello, world. This is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test.']


In [7]:
result = [item for item in result if item.strip()]
print(result)

['Hello,', 'world.', 'This', 'is', 'a', 'test.']


- 处理其他无意义字符

In [97]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


![Alt text](../../../img/LLM/ch01/tokenization_of_raw_text.png)

In [11]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 计算总的token长度

In [12]:
print(len(preprocessed))

4690


# 3、将tokens转换为token IDs

接下来,将文本tokens转换为token IDs,以便于之后我们可以通过embedding层对其进行处理

![Alt text](../../../img/LLM/ch01/token_2_tokenID.png)

从这些tokens,我们现在可以构建一个包括所有tokens的词表

In [13]:
# 进行去重
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [16]:
# 构建词表
vocab = {token:integer for integer,token in enumerate(all_words)}

In [17]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


构建一个tokenizer类

In [33]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        '''
        Tokenizer类
        :param vocab: 词表 
        '''
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

encode函数将词表转换为token IDs
decode函数将token IDs转换回文本

![Alt text](../../../img/LLM/ch01/encode_decode.png)

- 我们可以使用tokenizer来将文本转换为整数,这些整数可以作为LLM的embedding输入层

In [36]:
tokenizer = SimpleTokenizerV1(vocab)
text = "It's the last he painted, you know, Mrs. Gisburn said with pardonable pride."
ids = tokenizer.encode(text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 67, 7, 38, 851, 1108, 754, 793, 7]


- 将token IDs decode回文本

In [38]:
tokenizer.decode(ids)

"It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride."

In [39]:
tokenizer.decode(tokenizer.encode(text))

"It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride."

# 4、加入特殊文本tokens

1、为未知单词添加一些“特殊”标记并表示文本的结尾是很有用的

![Alt text](../../../img/LLM/ch01/special_token.png)

2、一些 tokenizer 使用特殊的 tokens 来帮助LLM提供额外的上下文
3、其中一些特殊 tokens 是
    3.1、[BOS]（序列的开始）标记文本的开始
    3.2、[EOS]（序列结束）标记文本的结束位置（这通常用于连接多个不相关的文本，例如，两篇不同的维基百科文章或两本不同的书，等等）
    3.3、[PAD]（填充）如果我们训练批量大小大于1的LLM（我们可能包括多个不同长度的文本；使用填充 token，我们将较短的文本填充到最长的长度，使所有文本的长度相等）
    3.4、[UNK]表示未包含在词汇表中的词汇
4、请注意，GPT-2不需要上述任何tokens，而只使用<|endoftext|> token 来降低复杂性
    4.1、<|endoftext|>类似于上面提到的[EOS] token
5、GPT还使用<|endoftext|>进行填充（因为我们在对批处理输入进行训练时通常使用掩码，所以无论如何我们都不会使用填充的tokens，所以这些tokens是什么并不重要）
6、GPT-2不对词汇表外的单词使用<UNK>标记；相反，GPT-2使用了一个字节对编码（BPE）标记器，它将字分解为子字单元，我们将在后面的部分中讨论

![Alt text](../../../img/LLM/ch01/endoftext_example.png)

In [40]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 由于词汇表中未包含“Hello”一词，因此上述操作会产生错误
- 为了处理这种情况，我们可以向词汇表中添加特殊的标记，如“<|unk|>”，以表示未知单词
- 由于我们已经在扩展词汇表，让我们添加另一个名为“<|endoftext|>”的标记，它在GPT-2训练中用于表示文本的末尾（它也用于连接的文本之间，例如如果我们的训练数据集由多篇文章、书籍等组成）

In [42]:
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [44]:
len(vocab.items())

1161

In [45]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1156)
('your', 1157)
('yourself', 1158)
('<|endoftext|>', 1159)
('<|unk|>', 1160)


- 我们还需要相应地调整tokenizer，以便它知道何时以及如何使用新的＜unk＞ token

In [46]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        '''
        Tokenizer类
        :param vocab: 词表 
        '''
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- 用修正后的tokenizer尝试一下

In [47]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [48]:
tokenizer.encode(text)

[1160,
 5,
 362,
 1155,
 642,
 1000,
 10,
 1159,
 57,
 1013,
 981,
 1009,
 738,
 1013,
 1160,
 7]

In [49]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

# 5、BytePair编码

- GPT-2使用字节对编码（BPE）作为其标记器
- 它允许模型将不在其预定义词汇表中的单词分解为更小的子单词单元甚至单个字符，使其能够处理词汇表外的单词
- 例如，如果GPT-2的词汇表中没有单词“unmiliarword”，它可能会将其标记为[“unfam”、“iliar”、“word”]或其他一些子单词细分，这取决于其经过训练的BPE合并
- 原始的BPE标记器可以在这里找到：https://github.com/openai/gpt-2/blob/master/src/encoder.py
- 在本章中，我们将使用OpenAI开源tiktoken库中的BPE标记器，该库在Rust中实现其核心算法，以提高计算性能
- 我创建了一个笔记本/并排比较这两种实现的bytepair_encoder（tiktoken比示例文本快5倍）

In [50]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [51]:
tokenizer = tiktoken.get_encoding("gpt2")

In [52]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [53]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE标记器将未知单词分解为子单词和单个字符：

![Alt text](../../../img/LLM/ch01/BPE_tokenizers.png)

# 6、滑动窗口数据采样

我们训练LLM一次生成一个单词，因此我们希望相应地准备训练数据，其中序列中的下一个单词表示要预测的目标：

![Alt text](../../../img/LLM/ch01/sliding_window.png)

In [54]:
with open("../../../Data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- 对于每个文本块，我们需要输入和目标
- 由于我们希望模型预测下一个单词，因此目标是向右移动一个位置的输入

In [63]:
enc_sample = enc_text[50:]

In [64]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f'x: {x}')
print(f'y:     {y}')

x: [290, 4920, 2241, 287]
y:     [4920, 2241, 287, 257]


一个接一个，预测如下：

In [65]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    
    print(context, '------>', desired)

[290] ------> 4920
[290, 4920] ------> 2241
[290, 4920, 2241] ------> 287
[290, 4920, 2241, 287] ------> 257


In [66]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 在我们介绍了注意力机制之后，我们将在后面的章节中讨论下一个单词的预测
- 目前，我们实现了一个简单的数据加载器，它在输入数据集上迭代，并返回偏移了一个的输入和目标

In [67]:
import torch

print(torch.__version__)

2.3.1+cpu


我们使用滑动窗口方法，将位置更改+1：

![Alt text](../../../img/LLM/ch01/sliding_window_example.png)

创建数据集和数据加载器，从输入文本数据集中提取块

In [76]:
from torch.utils.data import DataLoader, Dataset

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        '''
        :param txt: 文本 
        :param tokenizer: 
        :param max_length: 滑动窗口最大长度 
        :param stride: 滑动步长
        '''
        self.input_ids = []
        self.tar_ids = []
        
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            tar_chunk = token_ids[i + 1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.tar_ids.append(torch.tensor(tar_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.tar_ids[idx]

In [78]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=0
    )
    return dataloader

让我们针对上下文大小为4的LLM测试批处理大小为1的数据加载器：

In [79]:
with open(r'../../../Data/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

In [80]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [81]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


使用等于上下文长度的步幅（此处：4）的示例如下所示：

![Alt text](../../../img/LLM/ch01/sliding_window_4_stride.png)

我们还可以创建批处理输出
请注意，我们在这里增加步长，这样批次之间就不会有重叠，因为更多的重叠可能会导致过拟合增加

In [82]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


# 7、 创建token embeddings向量

数据已经准备好进行LLM
但最后让我们使用嵌入层将标记嵌入到连续向量表示中
通常，这些嵌入层是LLM本身的一部分，并在模型训练期间进行更新（训练）

![Alt text](../../../img/LLM/ch01/GPT_pipeline.png)

假设我们有以下四个输入示例，输入ID分别为5、1、3和2（标记化后）：

In [83]:
input_ids = torch.tensor([2, 3, 5, 1])

为了简单起见，假设我们的词汇量很小，只有6个单词，并且我们想要创建大小为3的嵌入：

In [84]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

这将产生6x3的权重矩阵：

In [87]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 对于那些熟悉一个热编码的人来说，上面的嵌入层方法本质上只是在完全连接层中实现一个热编码器-然后进行矩阵乘法的更有效的方式，这在中的补充代码中进行了描述
- 因为嵌入层只是一种更有效的实现，相当于一个热编码和矩阵乘法方法，所以它可以被视为一个可以通过反向传播优化的神经网络层
- 要将id为3的token转换为三维向量，我们执行以下操作：

In [88]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


注意,上面是embedding_layer权重矩阵中的第4行
要嵌入上面的所有四个input_ids值，我们需要进行以下操作

In [89]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


嵌入层本质上是一种查找操作：

![Alt text](../../../img/LLM/ch01/embedding_layer.png)

# 8、编码单词位置

- 嵌入层将ID转换为相同的矢量表示，而不考虑它们在输入序列中的位置：

![Alt text](../../../img/LLM/ch01/embedding_layer_example.png)

- 位置嵌入与标记嵌入向量相结合，以形成大型语言模型的输入嵌入：

![Alt text](../../../img/LLM/ch01/position_embedding.png)

- BytePair编码器的词汇表大小为50257：
- 假设我们想要将输入令牌编码为256维向量表示：

In [90]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 如果我们从数据加载器中采样数据，我们将每个批次中的token嵌入到256维向量中
- 如果我们有一个批量大小为8，每个有4个token，则会产生一个8 x 4 x 256的张量：

In [91]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [92]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [93]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


GPT-2使用绝对位置嵌入，因此我们只需创建另一个嵌入层：

In [1]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

NameError: name 'max_length' is not defined

In [95]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


要创建LLM中使用的输入嵌入，我们只需添加标记和位置嵌入：

In [96]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


在输入处理工作流的初始阶段，输入文本被分割成单独的标记
在该分割之后，这些token被转换为基于预定义词汇表的token ID：

![Alt text](../../../img/LLM/ch01/vocabulary_with_GPT_pipeline.png)